In [3]:
import numpy as np
from fluids import *


In [6]:
PM_OR = 0.049 # PM outer radius [m]
PM_IR = 0.045 # PM inner radius [m]
U_IR = 0.03 # Uranium layer inner radius [m]
U_rho = 8550 # Uranium mixture density [kg/m^3]

PM_thick = PM_OR - PM_IR # PM thickness [m]
L = 0.81 # CFE length [m]

# Define operating conditions
w = 7000 * 2*np.pi/60

P_centripetal = (2/3)*w**2*np.pi*U_rho*(PM_IR**3 - U_IR**3) / (2*np.pi*PM_IR) # Centripetal pressure from Uranium

f = H2(T=450, P=12.5e6) # Fluid state

P_out = 10e6 + P_centripetal # Outlet pressure [MPa]
mflow = 0.1 # Mass flow rate [kg/s]
rho = f.rho # Fluid density [kg/m^3]
Q = mflow / rho # Volumetric flow rate [m^3/s]
A = 2*np.pi*PM_OR*L # Cross sectional area [m^2]
q = Q / A # Darcian velocity [m/s]
mu = f.mu # Fluid viscosity [Pa * s]

k1 = 5.69771546674471E-12 # for 18% open porosity, 35% total
k2 = np.exp(-1.71588/(k1**0.08093)) # Correlation from Dey et al.

P_inlet = np.sqrt((mu*q/k1 + (rho*q**2)/k2) * 2*P_out*PM_thick + P_out**2) # Inlet pressure to PM [Pa]

PM_dP =  P_inlet - P_out # Pressure drop across PM [Pa]


print(f"T = 450K, P = 12.5 MPa\n")
print(f"rho = {rho}")
print(f"Q = {Q}")
print(f"A = {A}")
print(f"q = {q}")
print(f"mu = {mu}")
print(f"k1 = {k1}")
print(f"k2 = {k2}")
print(f"P_chamber = {P_out}")
print(f"P_inlet = {P_inlet}")
print(f"PM_dP = {PM_dP}")

T = 450K, P = 12.5 MPa

rho = 6.389293864533732
Q = 0.01565118182387713
A = 0.2493796248419578
q = 0.06276046743512399
mu = 1.1895048408744165e-05
k1 = 5.69771546674471e-12
k2 = 8.765844420214738e-07
P_chamber = 12182292.90313587
P_inlet = 12182931.8228505
PM_dP = 638.9197146296501
